## MSE 433 - Term Project

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from gurobipy import Model, GRB
from sklearn.ensemble import RandomForestRegressor

## Data Preprocessing

In [5]:
df = pd.read_csv('data/med_inv_dataset.csv')

df.columns = df.columns.str.lower()
df = df.dropna()
df['dateofbill'] = pd.to_datetime(df['dateofbill'])
df['month_name'] = df['dateofbill'].dt.strftime('%B') # Extract month name
df['month_number'] = df['dateofbill'].dt.month  # Extract month number
df['week_number'] = df['dateofbill'].dt.isocalendar().week  # Extract week number

# Create a bi-weekly period column
df['bi_weekly'] = (df['dateofbill'].dt.day - 1) // 14 + 1

df.sample(5)

,typeofsales,patient_id,specialisation,dept,dateofbill,quantity,returnquantity,final_cost,final_sales,rtnmrp,formulation,drugname,subcat,subcat1,month_name,month_number,week_number,bi_weekly
7186,Sale,12018092575,Specialisation20,Department1,2022-10-11,3,0,78.976,172.704,0.0,Form1,SODIUM CHLORIDE 0.9%,"IV FLUIDS, ELECTROLYTES, TPN",INTRAVENOUS & OTHER STERILE SOLUTIONS,October,10,41,1
13591,Sale,12018068231,Specialisation23,Department1,2022-09-14,1,0,284.160,325.000,0.0,Form1,EPOETIN BETA,INJECTIONS,CARDIIVASCULAR&HEMATOPOIETIC SYSTEM,September,9,37,1
6160,Sale,12018123972,Specialisation6,Department1,2022-12-27,3,0,66.880,183.678,0.0,Form1,AMOXYCILLIN 250MG + CLAVULANIC ACID 50MG,INJECTIONS,ANTI-INFECTIVES,December,12,52,2
3543,Sale,12018095061,Specialisation4,Department2,2022-02-08,5,0,96.516,344.500,0.0,Form1,ATRACURIUM BESYLATE 25MG/2.5ML,INJECTIONS,MUSCULO-SKELETAL SYSTEM,February,2,6,1
9687,Sale,12018090142,Specialisation8,Department1,2022-12-05,3,0,114.592,290.400,0.0,Form1,MULTIPLE ELECTROLYTES 500ML IVF,"IV FLUIDS, ELECTROLYTES, TPN",INTRAVENOUS & OTHER STERILE SOLUTIONS,December,12,49,1


In [6]:
df_bi_weekly = df.groupby(['subcat', 'month_name', 'month_number', 'bi_weekly'], as_index=False).agg(
    {
        'quantity': 'sum',
        'returnquantity': 'sum',
        'final_cost': 'sum',
        'final_sales': 'sum',
        'rtnmrp': 'sum'
    }
)

# Collect top 5 subcategories with the highest sum of quantity
top_5_subcats = df_bi_weekly.groupby('subcat')['quantity'].sum().nlargest(5).index

# Filter the dataframe for only the top 5 subcategories
filtered_top_5_per_subcat = df_bi_weekly[df_bi_weekly['subcat'].isin(top_5_subcats)]

In [8]:
filtered_df_bi_weekly = filtered_top_5_per_subcat.sort_values(by=['subcat', 'month_number', 'bi_weekly'])

# Add a biweekly index for every drugname in every subcat
filtered_df_bi_weekly['biweekly_index'] = (
    filtered_df_bi_weekly.groupby(['subcat'])
    .cumcount() + 1
)

# Filter rows where month_number is 4, 5, or 6 (2 and 3 is added just for their history but will get removed during the 
# feature engineering processs)
filtered_months_df = filtered_df_bi_weekly[filtered_df_bi_weekly['month_number'].isin([2, 3, 4, 5, 6])]

# The rest of the DataFrame but will remove these months later
rest_df = filtered_df_bi_weekly

In [9]:
def calculate_last_three_cycles(df, subcat, horizon = 'biweekly_index', quanity= 'quantity', train=False):
    ml_df = df[(df['subcat'] == subcat)][[horizon,'subcat', quanity]]

    ml_df['quantity_lastcycle']=ml_df[quanity].shift(+1)
    ml_df['quantity_2cycleback']=ml_df[quanity].shift(+2)
    ml_df['quantity_3cycleback']=ml_df[quanity].shift(+3)
    ml_df['quantity_4cycleback']=ml_df[quanity].shift(+4)
    ml_df['quantity_5cycleback']=ml_df[quanity].shift(+5)

    if train:
        ml_df = ml_df[~ml_df[horizon].isin([9, 10, 11, 12, 13, 14, 15, 16, 17])]

    ml_df = ml_df.dropna() #dropping na is necessary to avoid model failure other option

    X = ml_df[['subcat', horizon,'quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback', 'quantity_4cycleback', 'quantity_5cycleback']]
    y = ml_df[['subcat', quanity]]
    return X, y

In [10]:
trainX = pd.DataFrame(columns=['subcat', 'quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback', 
                               'quantity_4cycleback','quantity_5cycleback'])
trainY = pd.DataFrame(columns=['subcat', 'quantity'])

trainX_rtn = pd.DataFrame(columns=['subcat', 'quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback', 
                               'quantity_4cycleback','quantity_5cycleback'])
trainY_rtn = pd.DataFrame(columns=['subcat', 'returnquantity'])

list_subcat = filtered_top_5_per_subcat['subcat'].unique().tolist()

In [11]:
subcat_dict = {
    list_subcat[0]: {'Capacity': 200, 'shelf_life': (3,8), 'unit_cost': (40.85,322.27), 'salvage_value': (1,617.76)},
    list_subcat[1]: {'Capacity': 400, 'shelf_life': (2,5), 'unit_cost': (40.00,3178.00), 'salvage_value': (1,8014.0)},
    list_subcat[2]: {'Capacity': 100, 'shelf_life': (1,4), 'unit_cost': (40.00,3719.00), 'salvage_value': (1,4462.8)},
    list_subcat[3]: {'Capacity': 100, 'shelf_life': (1,3), 'unit_cost': (42.95,594.95), 'salvage_value': (1,327.11)},
    list_subcat[4]: {'Capacity': 300, 'shelf_life': (5,10), 'unit_cost': (40.00,3491.09), 'salvage_value': (1,1226.0)}
}

In [12]:
for key in list_subcat:
    X, y = calculate_last_three_cycles(rest_df, subcat = key, quanity = 'quantity', train=True)
    X_rtn, y_rtn = calculate_last_three_cycles(rest_df, subcat = key, quanity = 'returnquantity', train=True)
    trainX = pd.concat([X, trainX])
    trainY = pd.concat([y, trainY])
    trainX_rtn = pd.concat([X_rtn, trainX_rtn])
    trainY_rtn = pd.concat([y_rtn, trainY_rtn])

/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_67739/1466546704.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trainX = pd.concat([X, trainX])
/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_67739/1466546704.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trainX_rtn = pd.concat([X_rtn, trainX_rtn])


In [13]:
testX = pd.DataFrame(columns=['subcat', 'quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback',
                              'quantity_4cycleback','quantity_5cycleback'])
testY = pd.DataFrame(columns=['subcat', 'quantity'])

testX_rtn = pd.DataFrame(columns=['subcat', 'quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback', 
                               'quantity_4cycleback','quantity_5cycleback'])
testY_rtn = pd.DataFrame(columns=['subcat', 'returnquantity'])

In [14]:
for key in list_subcat:
    X, y = calculate_last_three_cycles(filtered_months_df, subcat = key, quanity = 'quantity')
    X_rtn, y_rtn = calculate_last_three_cycles(filtered_months_df, subcat = key, quanity = 'returnquantity')
    testX = pd.concat([X, testX])
    testY = pd.concat([y, testY])
    testX_rtn = pd.concat([X_rtn, testX_rtn])
    testY_rtn = pd.concat([y_rtn, testY_rtn])

/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_67739/307547127.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testX = pd.concat([X, testX])
/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_67739/307547127.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testX_rtn = pd.concat([X_rtn, testX_rtn])


## Predictive Analysis - Part 1

### Lets select one subcat here since we are trying to predict and optimize for one subcat at time

Originally we implemented this on streamlit which allowed the user to use a drop down to select a subcat.

In [18]:
i = 0
for subcate in list_subcat:
    print(f"Number {i}: {subcate}")
    i+=1

Number 0: INHALERS & RESPULES
Number 1: INJECTIONS
Number 2: IV FLUIDS, ELECTROLYTES, TPN
Number 3: LIQUIDS & SOLUTIONS
Number 4: TABLETS & CAPSULES


In [16]:
selected_subcat = list_subcat[1]
print(f"Selected subcategory: {selected_subcat}")

Selected subcategory: INJECTIONS


In [30]:
filtered_df = trainX[trainX['subcat'] == selected_subcat]
# selected_drugname = st.selectbox('Select a drug', filtered_df['drugname'].unique())

# Cost Coefficients
holding_cost = 2 # Cost per unit held
stockout_penalty = 50  # Cost per stockout
waste_penalty = 10 # Cost for expired stock

In [31]:
seed = 800

In [32]:
model = RandomForestRegressor(random_state=seed)
model_rtn = RandomForestRegressor(random_state=seed)

In [33]:
# This is for demand
tempx = trainX[(trainX['subcat'] == selected_subcat)].copy()
t_x = tempx[['quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback','quantity_4cycleback','quantity_5cycleback']]
t_y = trainY[(trainY['subcat']==selected_subcat)]['quantity']
model.fit(t_x, t_y)
test = testX[(testX['subcat'] == selected_subcat)][['quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback',
                                                                                            'quantity_4cycleback','quantity_5cycleback']]
predicted_demand = model.predict(test)
predicted_demand = np.ceil(predicted_demand).astype(int) 

In [34]:
predicted_demand

array([466, 481, 137, 471, 520, 162, 473, 377, 219])

In [35]:
# This is for return qty
tempx_rtn = trainX_rtn[(trainX_rtn['subcat'] == selected_subcat)].copy()
t_x = tempx_rtn[['quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback','quantity_4cycleback','quantity_5cycleback']]
t_y = trainY_rtn[(trainY_rtn['subcat']==selected_subcat)]['returnquantity']

model_rtn.fit(t_x, t_y)

test = testX_rtn[(testX_rtn['subcat'] == selected_subcat)][['quantity_lastcycle', 'quantity_2cycleback', 'quantity_3cycleback',
                                                                                                'quantity_4cycleback','quantity_5cycleback']]
predicted_return = model_rtn.predict(test)
predicted_return = np.ceil(predicted_return).astype(int) 

In [36]:
test_index = testX[(testX['subcat'] == selected_subcat)]['biweekly_index']
    
T = len(predicted_demand)-1

temp_dict = subcat_dict[selected_subcat]

# Create a DataFrame with biweekly_index and predicted_demand
prediction_df = pd.DataFrame({
    'biweekly_index': test_index,
    'Predicted_Demand': predicted_demand,
    'Return_Prediction': predicted_return,
    "Unit_Cost": np.random.uniform(temp_dict['unit_cost'][0], temp_dict['unit_cost'][1], len(predicted_demand)),  
    "Salvage_Value": np.random.uniform(temp_dict['salvage_value'][0], temp_dict['salvage_value'][1], len(predicted_demand)),
    "Shelf_Life": np.random.randint(temp_dict['shelf_life'][0], temp_dict['shelf_life'][1], len(predicted_demand))  
})

In [37]:
# Create a temporary table with biweekly_index, month_name, and bi_weekly
temp_table = filtered_df_bi_weekly[['biweekly_index', 'month_name', 'bi_weekly']].drop_duplicates()

# Create a temporary table with biweekly_index, month_name, and bi_weekly
temp_table = filtered_df_bi_weekly[['biweekly_index', 'month_name', 'bi_weekly']].drop_duplicates()

# Join prediction_df with the temp_table on biweekly_index
prediction_df = prediction_df.merge(temp_table, on='biweekly_index', how='left')
# Reorder columns to place month_name and bi_weekly at the beginning
columns_order = ['month_name', 'bi_weekly'] + [col for col in prediction_df.columns if col not in ['month_name', 'bi_weekly']]
prediction_df = prediction_df[columns_order]

# Rename 'bi_weekly' to 'cycle_number'
prediction_df.rename(columns={'bi_weekly': 'cycle_number'}, inplace=True)

# Drop the 'biweekly_index' column
prediction_df.drop(columns=['biweekly_index'], inplace=True)

prediction_df = prediction_df.reset_index(drop=True)

In [28]:
prediction_df

,month_name,cycle_number,Predicted_Demand,Return_Prediction,Unit_Cost,Salvage_Value,Shelf_Life
0,April,1,466,54,71.079450,5956.158609,2
1,April,2,481,68,753.246354,6349.414321,2
2,April,3,137,17,2214.642216,7783.717174,3
3,May,1,471,56,1510.863527,7966.760977,3
4,May,2,520,46,86.692612,6738.661872,2
5,May,3,162,9,852.201262,7653.595406,2
6,June,1,473,46,1208.981659,2050.843844,4
7,June,2,377,50,1927.979937,3547.379556,4
8,June,3,219,31,2320.159117,3207.245390,4
9,July,1,219,31,2320.159117,3207.245390,4


## Prescriptive Analysis

This is where Part 2 of the prescriptive model begins

In [38]:
# Gurobi Model
model = Model("Multi_Period_Medical_Inventory_Optimization")

# Decision Variables
Q = model.addVars(prediction_df.index, vtype=GRB.CONTINUOUS, name="OrderQty")  # Order quantity

# State Variables
I = model.addVars(prediction_df.index, vtype=GRB.CONTINUOUS, name="Inventory")  # Inventory level

# Auxiliary Variables (Derived)
Y = model.addVars(prediction_df.index, vtype=GRB.CONTINUOUS, name="Expired")   # Expired stock
S = model.addVars(prediction_df.index, vtype=GRB.CONTINUOUS, name="Stockout")  # Stockout

# Objective: Minimize Total Cost
model.setObjective(
    sum(prediction_df.loc[i, "Unit_Cost"] * Q[i] + 
        holding_cost * (I[i]) +
        stockout_penalty * S[i] + 
        waste_penalty * Y[i] - 
        prediction_df.loc[i, "Salvage_Value"] * prediction_df.loc[i, "Return_Prediction"]
        for i in prediction_df.index),
    GRB.MINIMIZE
)

In [ ]:
# Constraints:
for i in prediction_df.index:
    # Safety Stock Constraint
    safety_stock = 0.2 * prediction_df["Predicted_Demand"]  # Example: 20% of demand as buffer
    
    # Inventory Balance Constraint
    if i >= T:  # Ensure we don't reference out-of-bounds indices
        continue
    
    # Starting Inventory at capacity
    model.addConstr(I[0] == subcat_dict[selected_subcat]['Capacity'], name=f"Initial_Inventory{i}")
    
    # Ensure inventory level meets safety stock requirements
    model.addConstr(I[i] >= safety_stock[i], name=f"SafetyStock_{i}")
    
    model.addConstr(I[i+1] == I[i] + Q[i] + prediction_df.loc[i, "Return_Prediction"] - prediction_df.loc[i, "Predicted_Demand"] - Y[i], name=f"Inventory_Balance_{i}")

    model.addConstr(I[i+1] <= subcat_dict[selected_subcat]['Capacity'], name=f"Space_constraint{i}")
    
    # Expired Inventory Constraint
    model.addConstr(Y[i] <= I[i], name=f"Expiry_{i}")

    # Stockout Constraint
    model.addConstr(S[i] >= prediction_df.loc[i, "Predicted_Demand"] - I[i] - Q[i], name=f"Stockout_{i}")

In [40]:
# Solve Model
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.3.0 24D81)

CPU model: Intel(R) Core(TM) i7-8557U CPU @ 1.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40 rows, 40 columns and 88 nonzeros
Model fingerprint: 0xe94167fa
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+02]
Presolve removed 29 rows and 23 columns
Presolve time: 0.01s
Presolved: 11 rows, 17 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.2550960e+06   1.581000e+02   0.000000e+00      0s
       6   -5.2064684e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective -5.206468394e+05


In [41]:
prediction_df["Optimal_Order"] = [Q[i].x for i in prediction_df.index]
prediction_df["Inventory_Level"] = [I[i].x for i in prediction_df.index]
prediction_df["Expired_Stock"] = [Y[i].x for i in prediction_df.index]
prediction_df["Stockouts"] = [S[i].x for i in prediction_df.index]

In [45]:
prediction_df

,month_name,cycle_number,Predicted_Demand,Return_Prediction,Unit_Cost,Salvage_Value,Shelf_Life,Optimal_Order,Inventory_Level,Expired_Stock,Stockouts
0,April,1,466,54,1657.018954,1995.833679,2,0.0,812.0,0.0,0.0
1,April,2,481,68,1304.964561,3430.611893,4,227.2,400.0,0.0,0.0
2,April,3,137,17,1791.065171,2555.012774,4,0.0,214.2,0.0,0.0
3,May,1,471,56,485.870853,5692.207903,4,720.8,94.2,0.0,0.0
4,May,2,520,46,2754.748307,5336.359449,3,106.4,400.0,0.0,13.6
5,May,3,162,9,836.301876,3496.187609,4,215.2,32.4,0.0,0.0
6,June,1,473,46,339.400138,5629.673984,4,659.4,94.6,0.0,0.0
7,June,2,377,50,2652.499942,7065.367510,4,0.0,327.0,0.0,50.0
8,June,3,219,31,704.250327,4511.774910,4,0.0,0.0,0.0,0.0
9,July,1,219,31,704.250327,4511.774910,4,0.0,0.0,0.0,0.0


Each cycle represents a bi-weekly period.

In [42]:
fig1 = px.line(
        prediction_df,
        x=prediction_df.index,
        y=['Predicted_Demand', 'Return_Prediction'],
        labels={'value': 'Quantity', 'index': 'Cycles'},
        title="Predicted Demand and Return Prediction"
        )
# st.plotly_chart(fig1)
fig1.show()

Order Quantity Across Planning Period 

In [43]:
fig3 = px.bar(
            prediction_df,
            x=prediction_df.index,
            y='Optimal_Order',
            labels={'Optimal_Order': 'Order Quantity', 'index': 'Cycles'},
            title="Inventory Level Over Time"
        )
# Update layout to show all x-axis labels
fig3.update_layout(
    xaxis=dict(
        tickmode='linear'  # Ensures all x-axis labels are shown
    ))

fig3.show()

Inventory Level Over Time

In [44]:
fig2 = px.line(
            prediction_df,
            x=prediction_df.index,
            y='Inventory_Level',
            labels={'Inventory_Level': 'Inventory Level', 'index': 'Cycles'},
            title="Inventory Level Over Time"
        )

fig2.show()